In [71]:
import json
import websocket
import pandas as pd

# Creating a connection to serv


In [72]:
API_Key = "0sWK5vOVVm9PIiK6zWZCCQOc0RarZX8iHEbYD9MVw78q1XIp7fiowKhiOSCgIKnD"
Secret_Key = "LsWWxbgiIr029rD62I5QLeu4PTttFRgZ9WxSvPCsCS993JKGIOG9gsixvoZHnLyg"

In [73]:
from binance.client import Client
import pandas as pd

In [78]:
client = Client(api_key = API_Key, api_secret = Secret_Key, tld="com")


In [79]:
client.ping()

{}

In [80]:
account = client.get_account()

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [ ]:
account = client.get_account()
balances_df = pd.DataFrame(account["balances"])
balances_df

#Check binance trade fees


In [81]:
client.get_trade_fee(symbol="BTCUSDT")

BinanceAPIException: APIError(code=-2008): Invalid Api-Key ID.

In [ ]:
client.get_trade_fee(symbol="BUSDUSDT")

In [82]:
client.get_symbol_info(symbol= "BTCUSDT")

{'symbol': 'BTCUSDT',
 'status': 'TRADING',
 'baseAsset': 'BTC',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.01000000',
   'maxPrice': '1000000.00000000',
   'tickSize': '0.01000000'},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.00001000',
   'maxQty': '9000.00000000',
   'stepSize': '0.00001000'},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},
  {'filterType': 'MARKET_LOT_SIZE',
   'minQty': '0.00000000',
   'maxQty': '153.39618845',
   'stepSize': '0.00000000'},
  {'filterType': 'TRAILING_DELTA',
   'minTra

# Get Live prices

In [83]:
client.get_symbol_ticker(symbol = "BTCUSDT") # current price for one symbol

{'symbol': 'BTCUSDT', 'price': '39912.25000000'}

In [84]:
client.get_avg_price(symbol="BTCUSDT") # recent avg price for last 5 mins 

{'mins': 5, 'price': '39939.65012340', 'closeTime': 1706158200620}

In [85]:
float(client.get_symbol_ticker(symbol = "BTCUSDT")["price"])

39912.24

In [86]:
last24 = client.get_ticker(symbol= "BTCUSDT")
last24


{'symbol': 'BTCUSDT',
 'priceChange': '191.14000000',
 'priceChangePercent': '0.481',
 'weightedAvgPrice': '39978.95579901',
 'prevClosePrice': '39721.10000000',
 'lastPrice': '39912.25000000',
 'lastQty': '0.00014000',
 'bidPrice': '39912.24000000',
 'bidQty': '2.97229000',
 'askPrice': '39912.25000000',
 'askQty': '6.67811000',
 'openPrice': '39721.11000000',
 'highPrice': '40555.00000000',
 'lowPrice': '39484.19000000',
 'volume': '34802.07422000',
 'quoteVolume': '1391350586.95514710',
 'openTime': 1706071801165,
 'closeTime': 1706158201165,
 'firstId': 3387210192,
 'lastId': 3388769034,
 'count': 1558843}

In [87]:
day_price_change = last24['priceChange']
day_price_change

'191.14000000'

# Getting Historical Data (OHLC & Volume)

In [88]:
timestamp = client._get_earliest_valid_timestamp(symbol = "BTCUSDT", interval = "1d")
timestamp

1502928000000

In [89]:
pd.to_datetime(timestamp, unit="ms") # earlist data from binance

Timestamp('2017-08-17 00:00:00')

In [90]:
def get_history(symbol, interval, start, end= None):
    bars = client.get_historical_klines(symbol = symbol,
                                    interval = interval, start_str = start, end_str = end, limit=1000)
    historicalPrices = pd.DataFrame(bars)
    historicalPrices["Date"] = pd.to_datetime(historicalPrices.iloc[:,0], unit= "ms") 
    historicalPrices.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Clos Time",
                                "Quote Asset Volume", "Number of Trades", "Taker Buy Base Asset Volume",
                                "Taker Buy Quote Asset Volume","Ignore", "Date"]
    historicalPrices = historicalPrices[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    historicalPrices.set_index("Date", inplace=True)
    for column in historicalPrices.columns:
        historicalPrices[column] = pd.to_numeric(historicalPrices[column], errors = "coerce")
    
    return historicalPrices

In [91]:
df = get_history(symbol="BTCUSDT", interval="1d", start=timestamp)
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2024-01-21,41696.05,41881.39,41500.98,41580.33,11730.163010
2024-01-22,41580.32,41689.65,39431.58,39568.02,55426.199110
2024-01-23,39568.02,40176.74,38555.00,39897.60,57956.633510


# Most recent (last 2 hours) Intraday Data (1m)


In [92]:
from datetime import datetime, timedelta

In [93]:
now = datetime.now()
now

datetime.datetime(2024, 1, 25, 5, 50, 3, 566230)

In [94]:
two_hours_before = now - timedelta(hours=2)
str(two_hours_before)

'2024-01-25 03:50:03.566230'

In [95]:
last_2h_1m_df = get_history(symbol = "BTCUSDT", interval = "1m", start=str(two_hours_before))
last_2h_1m_df

,Open,High,Low,Close,Volume
Date,,,,,
2024-01-25 03:51:00,40012.96,40018.11,40006.67,40018.10,14.82376
2024-01-25 03:52:00,40018.11,40038.66,40018.10,40034.01,12.70625
2024-01-25 03:53:00,40034.01,40034.01,40008.87,40008.88,6.22232
2024-01-25 03:54:00,40008.87,40010.87,40006.67,40010.86,11.98654
2024-01-25 03:55:00,40010.86,40015.55,40002.00,40015.54,29.98746
2024-01-25 03:56:00,40015.55,40015.67,40015.54,40015.66,2.20770
2024-01-25 03:57:00,40015.67,40015.67,40012.00,40012.00,5.57472
2024-01-25 03:58:00,40012.01,40012.01,40012.00,40012.01,1.02796
2024-01-25 03:59:00,40012.00,40012.01,40012.00,40012.01,1.55308


# Option 1: ThreadedWebsocketManager (only executable in scripts!)

In [96]:
from binance import ThreadedWebsocketManager
import time

In [97]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Coplete"])
df

,Open,High,Low,Close,Volume,Coplete


In [98]:
def stream_candles(msg):
    
    #extract required candle items
    event_time = pd.to_datetime(msg["E"], unit="ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
    first      = float(msg["k"]["o"])
    high      = float(msg["k"]["h"])
    low      = float(msg["k"]["l"])
    close      = float(msg["k"]["c"])
    volume      = float(msg["k"]["v"])
    complete      = msg["k"]["x"]
    
    # print out
    print("Time: {} | Price: {}".format(event_time, close))
    
    # feed df (add a new bar / update)
    df.loc[start_time] = [first,high,low,close,volume,complete]
    

In [99]:
# initialize the start WebSocket

twm = ThreadedWebsocketManager(tld="us")
twm.start()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\matei\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\matei\anaconda3\Lib\site-packages\binance\threaded_stream.py", line 59, in run
    self._loop.run_until_complete(self.socket_listener())
  File "C:\Users\matei\anaconda3\Lib\asyncio\base_events.py", line 629, in run_until_complete
    self._check_running()
  File "C:\Users\matei\anaconda3\Lib\asyncio\base_events.py", line 588, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
C:\Users\matei\anaconda3\Lib\threading.py:1040: RuntimeWarning: coroutine 'ThreadedApiManager.socket_listener' was never awaited
  self._invoke_excepthook(self)


In [ ]:
twm.start_symbol_miniticker_socket(callback = stream_data, symbol = "BTCUSDT")

In [ ]:
While True:
    time.sleep(20)
    twm.stop()
    break

# Option 2: BinanceSocketManager (executable in Jupyter!)

In [ ]:
import asyncio
from binance import AsyncClient, BinanceSocketManager

In [ ]:
def stream_data(msg):
    print(msg)

In [ ]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.symbol_miniticker_socket(symbol="BTCUSDT")

    async with ts as tscm:  # Add a colon here
        res = await tscm.recv()
        stream_data(res)

    await client.close_connection()

In [ ]:
n = 10


In [ ]:
await main()

# Creating a Test Order

In [ ]:
from binance.client import Client
import pandas as pd 

In [ ]:
# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(symbol="BTCUSDT", side = "BUY", type="MARKET", quantity=0.1)

In [ ]:
## Paper Trading in the Spot Testnet

# Paper Trading with the Spot Testnet API

In [ ]:
api_key = "VxuqPa3p6RrAN8wBxSP7BFcFr1xyA6Wc9FrWdOu6lFMANIlr5UhekZdep67NBgNJ"
secret_key = "EgB2E9eq55Kkjp6SJTpU4GK0Yp0SMqNyLdoHyC9cdbakF6BFTGonfloDADbGJYVb"

In [ ]:
client = Client(api_key = api_key, api_secret = secret_key, tld="com", testnet=True)

In [ ]:
test_account = client.get_account()

In [ ]:
import time
gt = client.get_server_time()
print(gt)
print(time.localtime())
aa = str(gt)
bb = aa.replace("{'serverTime': ","")
aa = bb.replace("}","")
gg=int(aa)
ff=gg-10799260
uu=ff/1000
yy=int(uu)
tt=time.localtime(yy)
print(tt)

In [ ]:
df = pd.DataFrame(test_account["balances"])
df

In [ ]:
client.get_asset_balance(asset="BTC")

In [ ]:
client.get_asset_balance(asset="USDT")

# Place a buy Order

In [ ]:
order = client.create_order(symbol="BTCUSDT", side= "SELL", type="MARKET", quantity=0.1)

In [ ]:
order

In [ ]:
df = pd.DataFrame(order["fills"])
df

In [ ]:
num_columns = ["price", "qty", "commission"]

In [ ]:
for column in num_columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")

In [ ]:
df.info()

In [ ]:
df.qty.sum()

In [ ]:
df.price.mul(df.qty).sum()

# Place Limit Order

In [ ]:
client.get_symbol_ticker(symbol="BTCUSDT")

In [ ]:
order1 = client.create_order(symbol="BTCUSDT", side="BUY", type="LIMIT",
                             quantity=0.1, timeInForce="GTC", price = 42000)

In [ ]:
order1

In [ ]:
id_buy_limit1 = order1["orderId"]
id_buy_limit1

In [ ]:
client.get_order(symbol = "BTCUSDT", orderId = id_buy_limit1)

In [ ]:
order2 = client.create_order(symbol="BTCUSDT", side="SELL", type="LIMIT",
                             quantity=0.1, timeInForce="GTC", price = 43000)

In [ ]:
id_sell_limit1 = order2["orderId"]


In [ ]:
open_orders = client.get_open_orders(symbol="BTCUSDT")
pd.DataFrame(open_orders)

In [ ]:
result = client.cancel_order(symbol = "BTCUSDT", orderId = id_sell_limit1)

In [ ]:
all_orders = client.get_all_orders(symbol="BTCUSDT")
pd.DataFrame(all_orders)

In [ ]:
trades = client.get_my_trades(symbol="BTCUSDT")
trades_df = pd.DataFrame(trades)

In [ ]:
trades_df.time = pd.to_datetime(trades_df.time, unit="ms", errors = "coerce")


In [ ]:
trades_df

# First very simple Trading Bot 

In [ ]:
# Condition
40210 % 10 

In [ ]:
from binance import ThreadedWebsocketManager

In [ ]:
twm = ThreadedWebsocketManager()
twm.start()

In [ ]:
def simple_bot(msg):

    time = pd.to_datetime(msg["E"], unit="ms")
    price = float(msg["c"])

    print("Time: {} | Price: {}".format(time, price))

    if int(price) % 10 == 0:
        order = client.create_order(symbol="BTCUSDT", side="BUY", type="MARKET", quantity=0.1)
        print("\n" + 50 * "-")
        print("Buy {} BTC for {} USDT".format(order["executedQty"], order["cummulativeQuoteQty"]))
        print(50 * "-" + "\n")
 
        twm.stop() # stop defined inside callback

In [ ]:
twm.start_symbol_miniticker_socket(callback = simple_bot, symbol = "BTCUSDT")
twm.join() # required if stop is defined in callback function